# VacationPy
----

### Observable Trends and Insights 
#### In this task, I understand that creating an exploratory-interactive geospatial, Weather information, heat maps, and visualization through Google API, and other similar API applications play a remarkable role to exploit the potential socio-economic value buried inside monotonously large datasets.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame

weather_csv_file = "../WeatherPy/output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pierre,44.37,-100.35,77.00,47.0,40.0,13.87,US,1.592704e+09
1,salalah,17.02,54.09,82.40,88.0,75.0,4.70,OM,1.592704e+09
2,kununurra,-15.77,128.73,84.20,30.0,31.0,13.87,AU,1.592704e+09
3,upernavik,72.79,-56.15,39.76,73.0,0.0,8.52,GL,1.592704e+09
4,seybaplaya,19.65,-90.67,86.00,84.0,40.0,11.41,MX,1.592704e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "lng"]].astype(float)

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df



,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,mirnyy,62.54,113.96,75.20,36.0,0.0,4.72,RU,1.592704e+09
77,sayat,38.78,63.88,71.60,40.0,0.0,9.17,TM,1.592704e+09
119,florianopolis,-27.61,-48.50,73.99,88.0,0.0,5.82,BR,1.592704e+09
143,birao,10.28,22.79,76.32,76.0,0.0,7.38,CF,1.592704e+09
149,saquena,-4.67,-73.52,71.58,95.0,0.0,0.87,PE,1.592704e+09
177,komsomolskiy,40.43,71.72,71.60,30.0,0.0,6.93,UZ,1.592704e+09
221,aksu,41.12,80.26,75.72,20.0,0.0,5.12,CN,1.592704e+09
233,shache,38.42,77.24,78.62,16.0,0.0,6.40,CN,1.592704e+09
275,itupiranga,-5.13,-49.33,77.00,88.0,0.0,2.19,BR,1.592704e+09
279,marrakesh,31.63,-8.01,73.40,43.0,0.0,3.36,MA,1.592704e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 11: mirnyy.
Closest hotel is Гостиница "УЮТ".
------------
Retrieving Results for Index 77: sayat.
Missing field/result... skipping.
------------
Retrieving Results for Index 119: florianopolis.
Closest hotel is Hotel Boutique Quinta das Videiras.
------------
Retrieving Results for Index 143: birao.
Missing field/result... skipping.
------------
Retrieving Results for Index 149: saquena.
Missing field/result... skipping.
------------
Retrieving Results for Index 177: komsomolskiy.
Closest hotel is Ikathouse.
------------
Retrieving Results for Index 221: aksu.
Closest hotel is Pudong Holiday Hotel.
------------
Retrieving Results for Index 233: shache.
Closest hotel is Delong Hotel.
------------
Retrieving Results for Index 275: itupiranga.
Closest hotel is JAIRO ORLINDO VIEIRA.
------------
Retrieving Results for Index 279: marrakesh.
Closest hotel is Les Jardins de La Koutoubia.
------------
Retrieving Results for Index 316: noumea.
Closest hotel is Hôte

In [9]:
hotel_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,mirnyy,62.54,113.96,75.20,36.0,0.0,4.72,RU,1.592704e+09,"Гостиница ""УЮТ"""
77,sayat,38.78,63.88,71.60,40.0,0.0,9.17,TM,1.592704e+09,NaN
119,florianopolis,-27.61,-48.50,73.99,88.0,0.0,5.82,BR,1.592704e+09,Hotel Boutique Quinta das Videiras
143,birao,10.28,22.79,76.32,76.0,0.0,7.38,CF,1.592704e+09,NaN
149,saquena,-4.67,-73.52,71.58,95.0,0.0,0.87,PE,1.592704e+09,NaN
177,komsomolskiy,40.43,71.72,71.60,30.0,0.0,6.93,UZ,1.592704e+09,Ikathouse
221,aksu,41.12,80.26,75.72,20.0,0.0,5.12,CN,1.592704e+09,Pudong Holiday Hotel
233,shache,38.42,77.24,78.62,16.0,0.0,6.40,CN,1.592704e+09,Delong Hotel
275,itupiranga,-5.13,-49.33,77.00,88.0,0.0,2.19,BR,1.592704e+09,JAIRO ORLINDO VIEIRA
279,marrakesh,31.63,-8.01,73.40,43.0,0.0,3.36,MA,1.592704e+09,Les Jardins de La Koutoubia


In [10]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))